# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constantes

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# Configura el entorno

load_dotenv()

# Carga variables de entorno 
# API Key OpenAI
api_key = os.getenv('OPENAI_API_KEY')
# Se comrpueba si la clave tiene un formato esperado, y en ese caso, se crea un objeto 
if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
    # Crea el objeto de OpenAI
    openai = OpenAI()
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
# API URL de Ollama y Headers
OLLAMA_API = os.getenv('OLLAMA_API')
HEADERS = os.getenv('OLLAMA_HEADERS')



La clave de API parece buena


In [4]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# prompts
system_prompt = "Eres un tutor especializado en desarrollo de aplicaciones en código Python, ingeniería de software, ciencia de datos, procesos de analítica de datos"
user_prompt = "Por favor, explica la siguiente pregunta sobre un código: " + question

In [6]:
# messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [7]:
# Haz que gpt-4o-mini responda, con streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

El código que presentaste utiliza la expresión `yield from`, junto con una comprensión de conjunto, para extraer y devolver una lista de autores de un conjunto de libros. Te explicaré cada parte del código:

1. **Comprensión de conjunto**: 
   python
   {book.get("author") for book in books if book.get("author")}
   
   Esta parte del código crea un conjunto (set) de autores. Aquí está desglosado:
   - `for book in books`: itera sobre una colección llamada `books`, que se supone que es una lista (o alguna otra colección) de diccionarios, donde cada diccionario representa un libro.
   - `book.get("author")`: extrae el valor asociado con la clave `"author"` de cada diccionario `book`. Este método se utiliza para evitar un `KeyError` en caso de que la clave no exista; retornará `None` en su lugar.
   - `if book.get("author")`: esta condición garantiza que solo se incluyan en el conjunto aquellos autores que no son `None` o que existen (es decir, los libros que efectivamente tienen un autor definido). 

   Como resultado, la comprensión de conjunto devolverá un conjunto de todos los autores distintos que están disponibles en la lista de libros.

2. **`yield from`**:
   python
   yield from <expresión>
   
   Esta parte indica que el código está dentro de una función generadora. La instrucción `yield from` se utiliza para delegar la producción de valores a otro generador o iterable. Al usar `yield from` con el conjunto de autores, el generador devolverá cada autor uno por uno a quien consume el generador, en vez de devolver todo el conjunto de una vez.

### Resumen
En resumen, el código hace lo siguiente:
- Extrae los autores de una lista de libros, asegurándose de que solo se incluyan los autores que existen (no `None`).
- Devuelve cada autor de forma individual utilizando un generador, lo que permite que el consumo de estos autores sea eficiente, ya que no se necesita crear una lista completa en memoria antes de devolver los resultados.

Este enfoque es útil para mantener bajo control el uso de memoria y permitir una procesamiento más fluido cuando se trabaja con grandes colecciones de datos.

In [8]:
# Haz que Llama 3.2 responda

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

¡Claro! Este código utiliza una combinación de técnicas de generación de datos y filtrado en Python.

**Análisis del código**

El código se compone de dos partes principales:

1. `yield from`: es una función que permite filtrar y generar los elementos de un iterador genérico.
2. `{book.get("author") for book in books if book.get("author")}`: es una expresión generadora que filtra y transforma los datos.

**¿Qué hace el código?**

El código busca todos los autores de libros en la lista `books` que tengan un valor válido para el campo `"author"`. El resultado se devuelve como una secuencia genérica que puede ser procesado por algún consumidor (por ejemplo, otro generador o una función).

Aquí hay una explicación paso a paso:

1. La expresión `{book.get("author") for book in books if book.get("author")}` es una generadora que:
	* Itera sobre la lista `books`.
	* Para cada libro (`book`), utiliza el método `get()` para obtener el valor del campo `"author"`. Si el campo no existe o no tiene un valor válido, se devuelve `None`.
2. El resultado de esta expresión es una secuencia genérica que contiene solo los valores válidos del campo `"author"`. Por ejemplo: `["John Doe", "Jane Smith"]`
3. La función `yield from` toma la secuencia genérica resultante y la transforma en un iterador genérico. Esto significa que, en lugar de devolver una lista o otro tipo de colección, se devuelve una estructura genérica que puede ser procesada para obtener los resultados finales.

**Por qué funciona el código?**

El código funciona porque utiliza técnicas de generación de datos y filtrado para crear un resultado eficiente. Al utilizar `yield from`, el código evita la necesidad de almacenar todos los valores en una lista intermedia, lo que puede ahorrar memoria y mejorar el rendimiento.

Además, al utilizar la expresión generadora `{book.get("author") for book in books if book.get("author")}`, el código filtra y transforma los datos de manera eficiente, sin necesidad de crear un iterador adicional o realizar operaciones adicionales.